### Web Scrapping

In [1]:
# !pip install --upgrade pip
# !pip install beautifulsoup4 --upgrade
# pip install lxml
# pip install html5lib

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
import glob

#### External HTML File From Browser Element Section 

In [2]:
lang = "urdu" # indonesian, turkish, russian, english, urdu, hindi, bangla
surah = "faatir"
file = f"{surah.lower()}_{lang.lower()}"

In [119]:
HTMLFile = open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Data/{file.lower()}.html", "r") # ayahs-block

In [120]:
content = HTMLFile.read()

In [121]:
soup = BeautifulSoup(content, 'lxml')

In [122]:
print(soup.prettify())

<html>
 <body>
  <div class="ayahs-block">
   <div class="ayah-1 ayah-single 35" id="1">
    <div class="row ayah-row">
     <div class="col-1 ayah-buttons">
      <span class="ayah-block-buttons ayah-number-button">
       Ayah 1
      </span>
      <span class="ayah-block-buttons play-ayah-button">
       Play Ayah
      </span>
      <span class="ayah-block-buttons stop-recitation-button" style="display: none;">
       Stop Recitation
      </span>
     </div>
     <div class="col-10 single-ayah 1">
      <span class="single-word" data-ts="" id="35-1-1">
       <span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">
        ٱلۡحَمۡدُ
       </span>
       <span class="word-transliteration">
        al-hamdu
       </span>
       <span class="word-translation font-urdu">
        سب تعریف
       </span>
      </span>
      <span class="single-word" data-ts="" id="35-1-2">
       <span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;

In [123]:
ayah_word = soup.findAll('span', attrs={"class":"single-word"})
ayah_word

[<span class="single-word" data-ts="" id="35-1-1"><span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">ٱلۡحَمۡدُ</span> <span class="word-transliteration">al-hamdu</span><span class="word-translation font-urdu">سب تعریف</span></span>,
 <span class="single-word" data-ts="" id="35-1-2"><span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">لِلَّهِ</span> <span class="word-transliteration">lillahi</span><span class="word-translation font-urdu">اللہ کے لیے ہے</span></span>,
 <span class="single-word" data-ts="" id="35-1-3"><span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">فَاطِرِ</span> <span class="word-transliteration">fatiri</span><span class="word-translation font-urdu">پیدا کرنے والا ہے</span></span>,
 <span class="single-word" data-ts="" id="35-1-4"><span class="word-arabic word-arabic-uthmani-hafs" style="display: inline-block;">ٱلسَّمَٰوَٰتِ</span> <span class="word-transliteration">al-samawati<

In [124]:
word_list = []
for word in ayah_word:
    var1 = word.find('span', attrs={"class":"word-arabic word-arabic-uthmani-hafs"})
    #try:
    #    var1_list = re.findall(r'''(?<=>)(.*)(?=</span>)''', var1)
    #    arabic=" ".join(var1_list)
    #except:
    #    pass
    #arabic = var1.get_text()
    arabic = var1.string
    var2 = word.find('span', attrs={"class":"word-transliteration"})
    transliterate = var2.string
    var3 = word.find('span', attrs={"class":"word-translation"})
    translate = var3.string
    word_list.append((arabic, transliterate, translate))

In [125]:
word_list

[('ٱلۡحَمۡدُ', 'al-hamdu', 'سب تعریف'),
 ('لِلَّهِ', 'lillahi', 'اللہ کے لیے ہے'),
 ('فَاطِرِ', 'fatiri', 'پیدا کرنے والا ہے'),
 ('ٱلسَّمَٰوَٰتِ', 'al-samawati', 'آسمانوں کا'),
 ('وَٱلۡأَرۡضِ', 'wal-ardi', 'اور زمین کا'),
 ('جَاعِلِ', "ja'ili", 'بنانے والا ہے'),
 ('ٱلۡمَلَـٰٓئِكَةِ', 'al-malaikati', 'فرشتوں کو'),
 ('رُسُلًا', 'rusulan', 'پیغام بر'),
 ('أُوْلِيٓ', 'uli', 'والے'),
 ('أَجۡنِحَةٖ', 'ajnihatin', 'پروں والے'),
 ('مَّثۡنَىٰ', 'mathna', 'دو ، دو'),
 ('وَثُلَٰثَ', 'wathulatha', 'اور تین، تین'),
 ('وَرُبَٰعَۚ', "waruba'a", 'اور چار، چار'),
 ('يَزِيدُ', 'yazidu', 'اضافہ کرتا ہے'),
 ('فِي', 'fi', 'میں'),
 ('ٱلۡخَلۡقِ', 'al-khalqi', 'پیدائش (میں)'),
 ('مَا', 'ma', 'جو'),
 ('يَشَآءُۚ', 'yashau', 'وہ چاہتا ہے'),
 ('إِنَّ', 'inna', 'بیشک'),
 ('ٱللَّهَ', 'al-laha', 'اللہ تعالیٰ'),
 ('عَلَىٰ', 'ala', 'پر'),
 ('كُلِّ', 'kulli', 'ہر'),
 ('شَيۡءٖ', 'shayin', 'چیز'),
 ('قَدِيرٞ', 'qadirun', 'قادر ہے'),
 ('مَّا', 'ma', 'جو'),
 ('يَفۡتَحِ', 'yaftahi', 'کھولتا ہے'),
 ('ٱللَّهُ', 'al-lahu', 'ا

In [126]:
df_word_list = pd.DataFrame(word_list)
df_word_list.rename(columns={0:"arabic", 1:"transliterate", 2:f"translate_{lang.lower()}"}, inplace=True)
# df_word_list = df_word_list.iloc[3:,] for all body element
df_word_list

,arabic,transliterate,translate_urdu
0,ٱلۡحَمۡدُ,al-hamdu,سب تعریف
1,لِلَّهِ,lillahi,اللہ کے لیے ہے
2,فَاطِرِ,fatiri,پیدا کرنے والا ہے
3,ٱلسَّمَٰوَٰتِ,al-samawati,آسمانوں کا
4,وَٱلۡأَرۡضِ,wal-ardi,اور زمین کا
...,...,...,...
770,فَإِنَّ,fa-inna,تو بیشک
771,ٱللَّهَ,al-laha,اللہ تعالیٰ
772,كَانَ,kana,ہے
773,بِعِبَادِهِۦ,bi'ibadihi,اپنے بندوں کے ساتھ


In [127]:
df_word_list.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_{lang.lower().capitalize()}.xlsx", index=False)

#### Merge All Translate

In [7]:
all_trans_file = glob.glob(f"{surah.lower().capitalize()}*")
all_trans_file

['Faatir_Arabic_Transliterate_Translate_Words_Urdu.xlsx',
 'Faatir_Arabic_Transliterate_Translate_Words_English.xlsx',
 'Faatir_Arabic_Transliterate_Translate_Words_Hindi.xlsx',
 'Faatir_Arabic_Transliterate_Translate_Words_Indonesian.xlsx',
 'Faatir_Arabic_Transliterate_Translate_Words_Bangla.xlsx',
 'Faatir_Arabic_Transliterate_Translate_Words_Russian.xlsx',
 'Faatir_Arabic_Transliterate_Translate_Words_Turkish.xlsx']

In [23]:
df_master = pd.read_excel(all_trans_file[0])
for i in all_trans_file[1:]:
    df_var = pd.read_excel(f"{i}")
    #df_master = pd.merge(df_master, df_var, how="left", on=["arabic","transliterate"])
    df_master = pd.concat([df_master,df_var.iloc[:,2]], axis=1)
    #df_master = df_master.join(df_var, how="left", lsuffix=["arabic","transliterate"],rsuffix=["arabic","transliterate"])
    #df_master.drop_duplicates(inplace=True)
    df_master.reset_index(drop=True, inplace=True)
df_master

,arabic,transliterate,translate_urdu,translate_english,translate_hindi,translate_indonesian,translate_bangla,translate_russian,translate_turkish
0,ٱلۡحَمۡدُ,al-hamdu,سب تعریف,All praises,सब तारीफ़,segala puji,সকল প্রশংসা,Хвала,hamd olsun
1,لِلَّهِ,lillahi,اللہ کے لیے ہے,(be) to Allah,अल्लाह के लिए है,bagi Allah,আল্লাহরই জন্যে,"Аллаху,",Allah'a
2,فَاطِرِ,fatiri,پیدا کرنے والا ہے,Creator,पैदा करने वाला है,pencipta,(যিনি) স্রষ্টা,Творцу,yoktan var eden
3,ٱلسَّمَٰوَٰتِ,al-samawati,آسمانوں کا,(of) the heavens,आसमानों,langit(jamak),আকাশের,небес,gökleri
4,وَٱلۡأَرۡضِ,wal-ardi,اور زمین کا,and the earth,और ज़मीन का,dan bumi,ও পৃথিবীর,"и земли,",ve yeri
...,...,...,...,...,...,...,...,...,...
770,فَإِنَّ,fa-inna,تو بیشک,then indeed,तो बेशक,maka sesungguh-Nya,নিশ্চযই় তখন,"то, поистине,",kuşkusuz
771,ٱللَّهَ,al-laha,اللہ تعالیٰ,Allah,अल्लाह,Allah,আল্লাহ,Аллах,Allah
772,كَانَ,kana,ہے,is,है,adalah,হবেন,является,kullarını
773,بِعِبَادِهِۦ,bi'ibadihi,اپنے بندوں کے ساتھ,of His slaves,अपने बन्दों को,dengan hamba-hamba-Nya,তাঁর দাসদের সম্পর্কে,Своих рабов,*


In [ ]:
df_master.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx", index=False)